In [1]:
!pip install ultralytics
!pip install paddleocr
!pip install paddlepaddle -f https://www.paddlepaddle.org.cn/whl/paddle/linux/mkl/avx/stable.html

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.8/974.8 kB 26.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 5.0 MB/s eta 0:00:000:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found exi

In [2]:
import cv2
import numpy as np
from ultralytics import YOLO
from paddleocr import PaddleOCR
import re

# ---- Config ----
video_path = '/kaggle/input/yooooo/IMG_5778.mp4'
output_video_path = 'output_video.mp4'
output_txt_path = 'license_plates.txt'
#roi_points = np.array([[374, 692], [1790, 671], [1794, 880], [343, 908]])  # Replace with your ROI points
#roi_points = np.array([[334, 880], [348, 709], [1765, 720], [1769, 872]])  # Replace with your ROI points
roi_points = np.array([[1823, 1042], [269, 1014], [269, 1226], [1864, 1212]])  # Replace with your ROI points

# ---- Load YOLOv8 Model ----
model = YOLO("/kaggle/input/yooooo/best (3).pt")  # yolov8 format
model.conf = 0.5  # Confidence threshold

# ---- Initialize PaddleOCR ----
ocr = PaddleOCR(use_angle_cls=True, lang='en', use_gpu=True)

# ---- Video Input/Output Setup ----
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_size = (int(cap.get(3)), int(cap.get(4)))
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, frame_size)

# ---- Create ROI Mask ----
roi_mask = np.zeros((frame_size[1], frame_size[0]), dtype=np.uint8)
cv2.fillPoly(roi_mask, [roi_points], 255)

# ---- Output Text File Setup ----
text_output = open(output_txt_path, 'w')

frame_num = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_num += 1
    masked_frame = cv2.bitwise_and(frame, frame, mask=roi_mask)
    results = model(masked_frame)

    for r in results:
        if r.boxes is None:
            continue

        boxes = r.boxes.xyxy.cpu().numpy().astype(int)

        for box in boxes:
            x1, y1, x2, y2 = box

            # ---- Crop Detected License Plate ----
            cropped = frame[y1:y2, x1:x2]

            

            # ---- Preprocess for OCR ----
            gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            contrast = clahe.apply(gray)
            clean = cv2.bilateralFilter(contrast, 9, 75, 75)
            cropped_resized = cv2.resize(clean, (300, 100), interpolation=cv2.INTER_CUBIC)
            cropped_resized = cv2.cvtColor(cropped_resized, cv2.COLOR_GRAY2BGR)

            # ---- OCR ----
            ocr_results = ocr.ocr(cropped_resized, cls=True)
            best_text = ""
            best_conf = 0

            if ocr_results and isinstance(ocr_results, list):
                for result_line in ocr_results:
                    if result_line:
                        for res in result_line:
                            if res and len(res) >= 2:
                                text, confidence = res[1][0], res[1][1]
                                if confidence > best_conf:
                                    best_text = text
                                    best_conf = confidence

            # ---- Show Detection ----
            if best_conf > 0.3:
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, best_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                text_output.write(f"Frame {frame_num}: {best_text} (conf: {best_conf:.2f})\n")
                print(f"Detected on Frame {frame_num}: {best_text} (conf: {best_conf:.2f})")

    # Draw ROI
    cv2.polylines(frame, [roi_points], isClosed=True, color=(255, 0, 0), thickness=2)

    # Save processed frame
    out.write(frame)

# ---- Clean up ----
cap.release()
out.release()
text_output.close()

print("✅ Processing complete. Outputs saved.")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


/usr/local/lib/python3.11/dist-packages/paddle/utils/cpp_extension/extension_utils.py:711: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)


download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 3910/3910 [00:15<00:00, 248.07it/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv4/english/en_PP-OCRv4_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer/en_PP-OCRv4_rec_infer.tar


100%|██████████| 10000/10000 [00:18<00:00, 534.91it/s]


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2138/2138 [00:14<00:00, 142.68it/s]

[2025/04/15 17:44:02] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, use_mlu=False, use_gcu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_l


0: 544x800 (no detections), 40.2ms
Speed: 14.6ms preprocess, 40.2ms inference, 69.3ms postprocess per image at shape (1, 3, 544, 800)

0: 544x800 (no detections), 8.4ms
Speed: 3.7ms preprocess, 8.4ms inference, 0.6ms postprocess per image at shape (1, 3, 544, 800)

0: 544x800 (no detections), 8.4ms
Speed: 3.9ms preprocess, 8.4ms inference, 0.6ms postprocess per image at shape (1, 3, 544, 800)

0: 544x800 (no detections), 8.4ms
Speed: 3.9ms preprocess, 8.4ms inference, 0.6ms postprocess per image at shape (1, 3, 544, 800)

0: 544x800 (no detections), 8.8ms
Speed: 5.1ms preprocess, 8.8ms inference, 0.6ms postprocess per image at shape (1, 3, 544, 800)

0: 544x800 (no detections), 8.4ms
Speed: 3.7ms preprocess, 8.4ms inference, 0.6ms postprocess per image at shape (1, 3, 544, 800)

0: 544x800 (no detections), 8.5ms
Speed: 3.8ms preprocess, 8.5ms inference, 0.6ms postprocess per image at shape (1, 3, 544, 800)

0: 544x800 (no detections), 8.4ms
Speed: 3.7ms preprocess, 8.4ms inference, 0.

In [3]:
import re
from collections import defaultdict

# --- CONFIG ---
input_path = "/kaggle/working/license_plates.txt"
output_path = "cleaned_plates.txt"

def clean_plate_text(text):
    """Remove all non-alphanumeric characters (typically symbols)."""
    return re.sub(r'[^A-Za-z0-9]', '', text)

def process_file(path):
    with open(path, 'r') as f:
        lines = f.readlines()

    processed = []
    for line in lines:
        match = re.search(r'Frame (\d+): (.+?) \(conf: ([0-9.]+)\)', line)
        if match:
            frame = int(match.group(1))
            raw_plate = match.group(2)
            conf = float(match.group(3))
            plate = clean_plate_text(raw_plate)
            processed.append((frame, plate, conf))

    return processed

def fix_split_plates(processed):
    combined = []
    i = 0
    while i < len(processed):
        frame, plate, conf = processed[i]
        if len(plate) == 2:
            while i + 1 < len(processed) and len(processed[i + 1][1]) == 2:
                i += 1
                frame, plate, conf = processed[i]  # dump previous, keep new
            if i + 1 < len(processed) and len(processed[i + 1][1]) == 5:
                five_frame, five_plate, five_conf = processed[i + 1]
                full_plate = plate + five_plate
                combined.append((five_frame, full_plate, five_conf))
                i += 2
                continue
        elif len(plate) == 7:
            combined.append((frame, plate, conf))
        i += 1
    return combined

def deduplicate(plates):
    plate_dict = defaultdict(list)
    for frame, plate, conf in plates:
        plate_dict[plate].append((conf, frame))

    final_results = {}
    for plate, entries in plate_dict.items():
        entries.sort(reverse=True)  # Sort by confidence descending
        main_conf, main_frame = entries[0]
        alts = entries[1:3]  # Up to 2 alternatives
        final_results[plate] = {
            'conf': main_conf,
            'frame': main_frame,
            'alt': alts
        }
    return final_results

def save_output(final_results, path):
    with open(path, 'w') as f:
        for plate, data in final_results.items():
            f.write(f"{plate} (conf: {data['conf']:.2f}, frame: {data['frame']})\n")
            for idx, (alt_conf, alt_frame) in enumerate(data['alt'], 1):
                f.write(f"  ALT{idx}: (conf: {alt_conf:.2f}, frame: {alt_frame})\n")
            f.write("\n")

# --- MAIN EXECUTION ---
raw_data = process_file(input_path)
combined_plates = fix_split_plates(raw_data)
deduped = deduplicate(combined_plates)
save_output(deduped, output_path)

print("✅ Done. Cleaned and deduplicated plates saved to:", output_path)


✅ Done. Cleaned and deduplicated plates saved to: cleaned_plates.txt
